In [1]:
import requests
import base64
import json
import pandas as pd
import numpy as np
import os
from math import ceil
from collections import defaultdict
import os
import yaml

In [2]:
URL = "https://platform.jpmorgan.com/research/dataquery/api/v2"
class DataQueryInterface(object):

    def __init__(self, username: str, password: str,
                 crt: str = "api_macrosynergy_com.crt",
                 key: str = "api_macrosynergy_com.key"):

        self.auth = base64.b64encode(bytes(f'{username:s}:{password:s}',
                                           "utf-8")).decode('ascii')
        self.headers = {"Authorization": f"Basic {self.auth:s}"}
        self.base_url = URL
        self.cert = (crt, key)

    def _fetch_ts(self, params: dict, start_date: str = None, end_date: str = None, 
                  calendar: str = "CAL_ALLDAYS", frequency: str = "FREQ_DAY"):

        params["format"] = "JSON"
        params["start-date"] = start_date
        params["end-date"] = end_date
        params["calendar"] = calendar
        params["frequency"] = frequency
        
        endpoint = "/expressions/time-series"
        url = self.base_url + endpoint
        results = []

        with requests.get(url=url, cert=self.cert, headers=self.headers, params=params) as r:
            self.last_response = r.text
        response = json.loads(self.last_response)

        assert "instruments" in response.keys()
        results.extend(response["instruments"])
        
        return results
    
    def get_tickers(self, tickers, original_metrics, **kwargs):
        
        unique_tix = list(set(tickers))
        dq_tix = []
        for metric in original_metrics:
            dq_tix += ["DB(JPMAQS," + tick + f",{metric})" for tick in unique_tix]
            

        no_tickers = len(dq_tix)
        iterations = ceil(no_tickers / 20)
        remainder = no_tickers % 20

        results = []
        tickers_copy = dq_tix.copy()
        for i in range(iterations):
            if i < (iterations - 1):
                tickers = tickers_copy[i * 20: (i * 20) + 20]
            else:
                tickers = tickers_copy[-remainder:]

            params = {"expressions": tickers}
            output = self._fetch_ts(params=params, **kwargs)
            results.extend(output)

        no_metrics = len(set([tick.split(',')[-1][:-1] for tick in tickers_copy]))

        output_dict = self.isolate_timeseries(results, original_metrics)
        df_column_wise = self.df_column(output_dict, original_metrics)
        
        return df_column_wise

        
    @staticmethod
    def isolate_timeseries(list_, metrics):

        output_dict = defaultdict(dict)
        size = len(list_)

        for i in range(size):
            try:
                r = list_.pop()
            except IndexError:
                break
            else:
                dictionary = r['attributes'][0]
                ticker = dictionary['expression'].split(',')
                metric = ticker[-1][:-1]

                ticker_split = ','.join(ticker[:-1])
                ts_arr = np.array(dictionary['time-series'])

                if ticker_split not in output_dict:
                    output_dict[ticker_split]['real_date'] = ts_arr[:, 0]
                    output_dict[ticker_split][metric] = ts_arr[:, 1]
                else:
                    output_dict[ticker_split][metric] = ts_arr[:, 1]


        return output_dict
    
    
    @staticmethod
    def df_column(output_dict, original_metrics):

        index = next(iter(output_dict.values()))['real_date']
        no_rows = index.size
        no_columns = len(output_dict.keys()) * len(original_metrics)
        arr = np.empty(shape=(no_rows, no_columns), dtype=np.float32)

        i = 0
        columns = []
        for metric in original_metrics:
            for k, v in output_dict.items():

                col_name = k + ',' + metric + ')'
                columns.append(col_name)
                arr[:, i] = v[metric]
                i += 1

        df = pd.DataFrame(data=arr, columns=columns)

        return df

In [3]:
def dq_multiple(tickers, metrics_=["value"], start_d="2000-01-01", bool=True):

    with open("config.yml", 'r') as f:
        cf = yaml.load(f, Loader=yaml.FullLoader)

    dq = DataQueryInterface(username=cf["dq"]["username"], password=cf["dq"]["password"],
                            crt="api_macrosynergy_com.crt",
                            key="api_macrosynergy_com.key")

    df_ts = dq.get_tickers(tickers=tickers, original_metrics=metrics_,
                           start_date=start_d)
    
    return df_ts

In [4]:
cids = ['AUD', 'CAD']

metrics = ['value', 'eop_lag', 'mop_lag']
tix_1 = [cid + '_CPIXFE_SJA_P6M6ML6AR' for cid in cids]

df = dq_multiple(tix_1, metrics, "2021-11-01")
print(df)


Number of metrics: 3.
    DB(JPMAQS,CAD_CPIXFE_SJA_P6M6ML6AR,value)  \
0                                    3.323818   
1                                    3.323818   
2                                    3.323818   
3                                    3.323818   
4                                    3.323818   
5                                         NaN   
6                                         NaN   
7                                    3.323818   
8                                    3.323818   
9                                    3.323818   
10                                   3.323818   
11                                   3.323818   
12                                        NaN   
13                                        NaN   
14                                   3.323818   

    DB(JPMAQS,AUD_CPIXFE_SJA_P6M6ML6AR,value)  \
0                                    1.820391   
1                                    1.820391   
2                                    1.820391 